<a href="https://colab.research.google.com/github/joampc/TFM2023/blob/main/notebooks/Food_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:

import pandas as pd
import csv






In [ ]:


""" lectura """
from google.colab import drive
drive.mount('/content/drive')

In [15]:
# Ir a Google Drive , hacer clic derecho sobre la carpeta compartida (Compartida conmigo o Shared with me) y seleccionar Organizar/Añadir acceso directo
# seleccionar en todas las ubicaciones Mi Unidad, y darle Añadir, de esta forma la carpeta TFM aparecera en la lista de carpetas del drive de Google y se podra acceder desde collab.
# COn este comando  lista todo dentro del drive
!ls "/content/drive/MyDrive"


 Alimentos	   'CV Formato Conciso.doc'	    Specs
'Colab Notebooks'  'Gastos Febrero.gsheet'	    TFM
 Colab_Notebooks    HeartDisease_prediction.ipynb


In [37]:
df_alimentos = pd.read_csv('/content/drive/MyDrive/TFM/Alimentos/listado_componentes.csv', delimiter='|', quoting=csv.QUOTE_NONE)
df_alimentos.head()

,id,Nombre,English Name,Nombre Cientifico,Codigo foodex,Parte Comestible,Descripcion
0,746,Aceite de algodón,"Cotton oil,",- () - (),100.0,NaN,NaN
1,747,Aceite de cacahuete,"Peanut oil,",- () - (),100.0,NaN,NaN
2,748,Aceite de coco,"Coconut oil,",- () - (),100.0,NaN,NaN
3,753,Aceite de colza,"Rape oil,",- () - (),100.0,NaN,NaN
4,749,Aceite de germen de trigo,"Wheat germ oil,",- () - (),100.0,NaN,NaN


In [38]:
#cantidad de alimentos
print(f'Número de filas: {df_alimentos.shape[0]}')


Número de filas: 1052


In [22]:
df0 = pd.read_csv('/content/drive/MyDrive/TFM/Alimentos/componentes_A.csv' , delimiter='|')
df0.head()

,ID,Grupo,Componente,Valor,Unidad,Fuente
0,746,Proximales,alcohol (etanol),0,g,310
1,746,Proximales,"energía, total",3700 (888),kJ (kcal),236
2,746,Proximales,"grasa, total (lipidos totales)",100,g,30
3,746,Proximales,"proteina, total",0,g,30
4,746,Proximales,agua (humedad),0,g,7


In [24]:
# uniendo todos los datasets de la composicion de alimentos
ruta_base = '/content/drive/MyDrive/TFM/Alimentos/componentes_'

# Lista para almacenar los DataFrames individuales
dfs = []

# Recorrer los archivos correspondientes a cada letra
for letra in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']:
    ruta_archivo = f"{ruta_base}{letra}.csv"
    df = pd.read_csv(ruta_archivo, delimiter='|')
    dfs.append(df)

# Combinar los DataFrames en uno solo
df_composicion = pd.concat(dfs)



# esta es la Información de composición (por 100 g de porción comestible)
df_composicion.head()

,ID,Grupo,Componente,Valor,Unidad,Fuente
0,746,Proximales,alcohol (etanol),0,g,310
1,746,Proximales,"energía, total",3700 (888),kJ (kcal),236
2,746,Proximales,"grasa, total (lipidos totales)",100,g,30
3,746,Proximales,"proteina, total",0,g,30
4,746,Proximales,agua (humedad),0,g,7


In [25]:
num_filas = df_composicion.shape[0]
print("Número de filas:", num_filas)

Número de filas: 39012


In [26]:
#el fichero grupo de alimentos puede servir para agrupar los ingredientes

In [28]:
df_grupos = pd.read_csv('/content/drive/MyDrive/TFM/Alimentos/grupos_alimentos.csv', delimiter='|')
df_grupos.head()

FileNotFoundError: ignored